### 

In [66]:
import numpy as np
import matplotlib.pyplot as plt

#### Generating Data for y = sin(5x)


In [67]:
def generateData():
    xstart=-2
    xend=2
    dx = 0.01
    coef=5

    nbPoints= int((xend-xstart)/dx)+1
    x = np.linspace(xstart, xend, nbPoints)
    y = np.sin(coef*x)
    return x,y

### The transformation
We have a non linear function $y = sin(ax)$, where we want to find a.

We get the linear relationship $z=ax$ where $z = arcsin(y)$, a becomes the slope.

For the sin fction $y ∈ [-1,1]$, and arcsine is defined on that domain.




In [68]:
def arcsin_transform(x,y):
    return np.arcsin(y)

Now we face a problem where the domain of the result (z in this case) of the transotmation is $[-π/2 ; π/2]$ (domain of arcsine).

$x ∈ [-2,2]=> 5x =z ∈ [-10,10]$ ,   to deal with this we introduce a method that detects discontnituities by locating  jump greater than π/2 from both sides  in consecutive z values. 

In [73]:
def z_correction(x, z_original):
    z = z_original.copy()
    
    for i in range(len(z)):
        # For each point, determine which "period" it should be in
        expected_value = 5 * x[i]  # What z should be if z = 5x
        
        # Find the closest multiple of π to add
        diff = expected_value - z[i]
        correction = round(diff / np.pi) * np.pi
        z[i] += correction
    
    return z

We now need to estimte the coeff 'a' from  $z = ax$ by using least squares linear regression.
to find $a$ we use the loss function $L(a)= \sum (z_i - ax_i)^2$

we expand: $ L(a) = \sum z_i^2 - 2az_ix_i + a^2x_i^2$

we set the derivative wrt a equal to zero: $ \frac{dL}{da} = 2ax_i^2 - 2z_ix_i = 0$

And so we aplly the fla $a = Σ(x_iz_i) / Σ(x_i²)$

In [74]:
def estimate_coefficient(x_data, z_data):
    numerator = np.sum(x_data * z_data)
    denominator = np.sum(x_data**2)
    coefficient = numerator / denominator
    
    return coefficient

#### Least Square Polynomial function from earlier exercice

In [75]:
def leastSquaresPolynomial(data, order):
    # Convert data to numpy array and extract x and y values
    data = np.array(data)
    x = data[:, 0]
    y = data[:, 1]
    
    n = len(x)
    k = order
    
    # Check if we have enough data points
    if n <= k:
        raise ValueError(f"Need more than {k} data points to fit polynomial of order {k}")
    
    # Construct the Vandermonde matrix A
    A = np.zeros((n, k + 1))
    for i in range(n):
        for j in range(k + 1):
            A[i, j] = x[i] ** j
    
    # Solve the normal equation: (A^T A) c = A^T y
    ATA = np.dot(A.T, A)
    ATy = np.dot(A.T, y)
    coefficients = np.linalg.solve(ATA, ATy)
    
    return coefficients

TEsting

In [76]:
x= generateData()[0]
y= generateData()[1]

z = z_correction(x,arcsin_transform(x,y))

transformed_data =[[x[i], z[i]] for i in range(len(x)) ]
coefficients = leastSquaresPolynomial(transformed_data, order=1)

In [82]:
print(f"Polynomial coefficients: [a0, a1] = {coefficients}")
print(f"Intercept a0 = {coefficients[0]:.6f}")
print(f"Slope a1 = {coefficients[1]:.6f}")

#Compare with direct analytical solution
print("\nAnalytical Solution")
print( f"a = {estimate_coefficient(x,z):.6f}")

#  Results
print(f"\nResults:")
print(f"true coef = 5")
print(f"leastSquaresPolynomial: {coefficients[1]:.6f}")
print(f"Direct analytical: {estimate_coefficient(x,z):.6f}")


Polynomial coefficients: [a0, a1] = [7.89330724e-17 4.96589237e+00]
Intercept a0 = 0.000000
Slope a1 = 4.965892

Analytical Solution
a = 4.965892

Results:
true coef = 5
leastSquaresPolynomial: 4.965892
Direct analytical: 4.965892
